In [1]:
"""
Coffee News Price Direction Pipeline
------------------------------------
This script performs the full workflow for:
1. Filtering coffee-related news articles
2. Merging with coffee price data
3. Generating manual and FinBERT-based labels
4. Merging and feature engineering
5. Modeling using classical ML (LogReg, RF, GBoost)
"""

import pandas as pd
import joblib
import numpy as np
import re
import nltk
import torch
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE

# 1. 뉴스 필터링
def filter_news_by_keywords(news_list, keywords):
    filtered_dfs = []
    for filename in news_list:
        df = pd.read_csv(f'./data/{filename}')
        df['is_price_related'] = df['title'].astype(str).apply(lambda x: any(k in x.lower() for k in keywords))
        filtered_dfs.append(df[df['is_price_related']])
    return pd.concat(filtered_dfs, ignore_index=True)

# 2. 수동 라벨링
def generate_manual_labels(news_df, price_df_path, output_path):
    coffee_price_df = pd.read_csv(price_df_path)
    news_df["date"] = pd.to_datetime(news_df["date"])
    coffee_price_df["Date"] = pd.to_datetime(coffee_price_df["Date"])
    coffee_price_df.sort_values("Date", inplace=True)
    coffee_price_df["prev_price"] = coffee_price_df["Coffee_Price"].shift(1)
    coffee_price_df["price_direction"] = coffee_price_df.apply(
        lambda row: "rise" if row["Coffee_Price"] > row["prev_price"]
        else "fall" if row["Coffee_Price"] < row["prev_price"]
        else "neutral", axis=1)
    merged_df = pd.merge(news_df, coffee_price_df[["Date", "price_direction"]],
                         left_on="date", right_on="Date", how="left")
    merged_df.rename(columns={"price_direction": "manual_price_direction"}, inplace=True)
    merged_df.drop(columns=["Date"], inplace=True)
    merged_df.to_csv(output_path, index=False)
    return merged_df

# 3. FinBERT 추론
def label_with_finbert(df, model_name="yiyanghkust/finbert-tone"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    titles = df["title"].astype(str).tolist()
    inputs = tokenizer(titles, padding=True, truncation=True, return_tensors="pt", max_length=128)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1).numpy()
    id2label = {0: "neutral", 1: "positive", 2: "negative"}
    label_map = {"positive": "rise", "negative": "fall", "neutral": "neutral"}
    df["price_direction"] = [label_map[id2label[p]] for p in preds]
    df.to_csv("./processed/finbert_coffee_news_labeled.csv", index=False)
    return df

# 4. 데이터 통합 및 전처리
def combine_and_engineer_features(finbert_df, manual_df):
    merged = pd.merge(finbert_df, manual_df, on=["date", "title", "url", "is_price_related"], how="left", suffixes=("_finbert", "_title"))
    merged["combined_price_direction"] = merged["price_direction"].fillna(merged["manual_price_direction"])
    merged.dropna(subset=["combined_price_direction"], inplace=True)

    nltk.download("stopwords")
    nltk.download("vader_lexicon")
    stop_words = set(stopwords.words("english"))
    analyzer = SentimentIntensityAnalyzer()

    def preprocess(text):
        text = re.sub(r"[^\w\s]", "", text.lower())
        return " ".join([word for word in text.split() if word not in stop_words])

    merged["preprocessed_title"] = merged["title"].astype(str).apply(preprocess)
    sentiments = merged["preprocessed_title"].apply(analyzer.polarity_scores)
    merged["positive_sentiment"] = sentiments.apply(lambda x: x["pos"])
    merged["negative_sentiment"] = sentiments.apply(lambda x: x["neg"])
    merged["neutral_sentiment"] = sentiments.apply(lambda x: x["neu"])

    price_keywords = ["rise", "increase", "jump", "surge", "climb", "fall", "drop", "decrease", "decline", "plunge"]
    for word in price_keywords:
        merged[f"{word}_present"] = merged["preprocessed_title"].str.contains(word).astype(int)

    merged["weighted_positive_sentiment"] = merged.apply(lambda x: x["positive_sentiment"] * 2 if x["combined_price_direction"] in ["rise", "fall"] else x["positive_sentiment"], axis=1)
    merged["weighted_negative_sentiment"] = merged.apply(lambda x: x["negative_sentiment"] * 2 if x["combined_price_direction"] in ["rise", "fall"] else x["negative_sentiment"], axis=1)
    return merged

# 5. 모델 학습 및 평가
def train_classical_models(df):
    features = ['preprocessed_title', 'positive_sentiment', 'negative_sentiment', 'neutral_sentiment'] +                [col for col in df.columns if '_present' in col] +                ['weighted_positive_sentiment', 'weighted_negative_sentiment']

    X = df[features]
    y = df["combined_price_direction"]
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(X["preprocessed_title"])
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
    X = pd.concat([X.drop("preprocessed_title", axis=1), tfidf_df], axis=1)

    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    le = LabelEncoder()
    y_encoded = le.fit_transform(y_resampled)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest": RandomForestClassifier(),
        "Gradient Boosting": GradientBoostingClassifier()
    }

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)
        print(f"Model: {name}")
        print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print(f"F1-score: {f1_score(y_test, y_pred, average='macro'):.4f}")
        try:
            print(f"AUC-ROC: {roc_auc_score(y_test, y_prob, multi_class='ovr'):.4f}")
        except:
            print("AUC-ROC could not be calculated")
        print(confusion_matrix(y_test, y_pred))
        print()

    joblib.dump(model, "./models/gradient_boosting_model.pkl")
    joblib.dump(tfidf, "./models/tfidf_vectorizer.pkl")
    joblib.dump(le, "./models/label_encoder.pkl")


In [2]:
def predict_new_data(news_df, model_path, vectorizer_path, label_encoder_path):
    import joblib
    import re
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    from nltk.corpus import stopwords
    import nltk

    # Load trained components
    model = joblib.load(model_path)
    vectorizer = joblib.load(vectorizer_path)
    label_encoder = joblib.load(label_encoder_path)

    # Preprocess
    nltk.download("vader_lexicon")
    nltk.download("stopwords")
    analyzer = SentimentIntensityAnalyzer()
    stop_words = set(stopwords.words("english"))

    def preprocess(text):
        text = re.sub(r"[^\w\s]", "", text.lower())
        return " ".join([word for word in text.split() if word not in stop_words])

    news_df["preprocessed_title"] = news_df["title"].astype(str).apply(preprocess)
    sentiments = news_df["preprocessed_title"].apply(analyzer.polarity_scores)
    news_df["positive_sentiment"] = sentiments.apply(lambda x: x["pos"])
    news_df["negative_sentiment"] = sentiments.apply(lambda x: x["neg"])
    news_df["neutral_sentiment"] = sentiments.apply(lambda x: x["neu"])

    price_keywords = ["rise", "increase", "jump", "surge", "climb", "fall", "drop", "decrease", "decline", "plunge"]
    for word in price_keywords:
        news_df[f"{word}_present"] = news_df["preprocessed_title"].str.contains(word).astype(int)

    news_df["weighted_positive_sentiment"] = news_df["positive_sentiment"]
    news_df["weighted_negative_sentiment"] = news_df["negative_sentiment"]

    # TF-IDF vectorize
    tfidf = vectorizer.transform(news_df["preprocessed_title"])
    tfidf_df = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names_out(), index=news_df.index)

    # Feature assembly
    basic_features = ['positive_sentiment', 'negative_sentiment', 'neutral_sentiment'] + \
                     [f"{w}_present" for w in price_keywords] + \
                     ['weighted_positive_sentiment', 'weighted_negative_sentiment']

    X = pd.concat([news_df[basic_features], tfidf_df], axis=1)

    # Predict
    y_pred_encoded = model.predict(X)
    y_pred = label_encoder.inverse_transform(y_pred_encoded)
    news_df["predicted_price_direction"] = y_pred

    return news_df


In [3]:
news_list = [
'dailycoffeenews_250331.csv',
'usda_coffee_articles_250520.csv',
'usda_robusta_articles_250520.csv',
'worldcoffeeportal_250331.csv'
]

price_keywords = [
"coffee", "arabica", "robusta", "futures", "coffee price", "commodity",
"coffee market", "crop", "harvest", "supply", "demand", "weather",
"drought", "frost", "yield", "shortage", "production", "export", "import",
"inventory", "logistics", "shipping", "rain", "climate"
]

for data in news_list:
    filtered_df = filter_news_by_keywords([data], price_keywords)
    filtered_df.to_csv(f"./processed/filtered_{data}", index=False)

    result_df = predict_new_data(
        filtered_df,
        model_path="./models/gradient_boosting_model.pkl",
        vectorizer_path="./models/tfidf_vectorizer.pkl",
        label_encoder_path="./models/label_encoder.pkl"
    )

    # 결과 확인
    print(result_df[["title", "predicted_price_direction"]].head())

    # 저장
    result_df.to_csv(f"./predictions/predict_{data}", index=False)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                               title predicted_price_direction
0  For Five Coffee Roasters Opens a Big One For A...                   neutral
1  Xanadu Coffee Closed ‘Until Further Notice’ as...                   neutral
2  Weekly Coffee News: Caffeine and Tinnitus + Re...                   neutral
3  Hutsul Coffee Factory Carries Ukrainian Qualit...                   neutral
4  Study Explores the ‘Arabica-Like’ Cup Qualitie...                   neutral


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                               title predicted_price_direction
0                       NASS 2025-01-24 15:00 Coffee                   neutral
1  FAS 2024-12-18 15:00 Coffee: World Markets and...                   neutral
2  Secretary Vilsack Highlights Historic Investme...                   neutral
3  USDA Highlights Success of Historic Partnershi...                   neutral
4  Coffee Is Hot Commodity, Even During This Reco...                      fall
                                               title predicted_price_direction
0  Secretary Rollins Suspends Live Animal Imports...                   neutral
1  Fact Sheet: Biden-Harris Administration Advanc...                   neutral
2  USDA Announces Grants and Technical Assistance...                   neutral
3  Biden-Harris Administration Announces $90 Mill...                   neutral
4  USDA Publishes Request for Information on the ...                   neutral


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                               title predicted_price_direction
0  Black Rifle Coffee Company appoints new Chief ...                   neutral
1  Costa Coffee appoints new exec to lead US trav...                   neutral
2  Celebrating 10 years of The Amsterdam Coffee F...                   neutral
3  Diversified store network yields strong profit...                   neutral
4  Jollibee Foods Corp outlines long-term coffee ...                   neutral
